# RRAP-IS Provenance Demo Notebook

> A tutorial of RRAP authentication using Jupyter notebooks.

- toc: true 
- badges: true
- comments: true
- categories: [jupyter, authentication]

## About

This notebook is a demonstration of authenticating with the RRAP-IS system.  

### Run all imports

Keep all your imports at the top of a notebook.  It allows for easier management.

In [8]:
import requests
import os
import sys
import json
import json2tree
from bs4 import BeautifulSoup
from IPython.display import IFrame, display, HTML, JSON, Markdown
from mdsisclienttools.auth.TokenManager import DeviceFlowManager

import networkx as nx
import nx_altair as nxa
from networkx.readwrite import json_graph

import warnings
warnings.filterwarnings(action='once')

### Define global variables

Similar to import we like to define notebook variable at the top and reuse them throughout the notebook

In [9]:
data_api = "https://data-api.testing.rrap-is.com"
registry_api = "https://registry-api.testing.rrap-is.com"
prov_api = "https://prov-api.testing.rrap-is.com"
auth_server = "https://auth.dev.rrap-is.com/auth/realms/rrap"
# garbage = "https://frogs.are.green"
base_urls = {'data_api': data_api, 'registry_api': registry_api, 'prov_api': prov_api, 'auth_server': auth_server}#, 'garbage': garbage}
display(f'Checking base urls')

for key, url in base_urls.items():
    try:
        print(f'Testing - {url}', end="")
        r = requests.get(url)
        r.raise_for_status()
        print(f' - Passed')
    except requests.exceptions.HTTPError as err:
        print(f' - Fail')
        raise SystemExit(err)
    except requests.exceptions.RequestException as e:
        # catastrophic error. bail.
        print(f' - Fail')
        raise SystemExit(e)

'Checking base urls'

Testing - https://data-api.testing.rrap-is.com - Passed
Testing - https://registry-api.testing.rrap-is.com - Passed
Testing - https://prov-api.testing.rrap-is.com - Passed
Testing - https://auth.dev.rrap-is.com/auth/realms/rrap - Passed


## Authentication

### Setup tokens using device authorisation flow against keycloak server

This could result in a browser window being opened if you don't have valid tokens cached in local storage.

[Return to Top](#toc)

In [10]:
# this caches the tokens
local_token_storage = ".tokens.json"

token_manager = DeviceFlowManager(
    stage="TEST",
    keycloak_endpoint=auth_server,
    local_storage_location=local_token_storage
)

Attempting to generate authorisation tokens.

Looking for existing tokens in local storage.

Validating found tokens

Found tokens valid, using.



In [11]:
def convert_json_to_html(json_input):
    out_file = open("example.json", "w")
    json.dump(json_input, out_file)
    out_file.close()
    !json2tree -j "example.json" -o "output.html" -t 1
    with open("output.html") as f:
        cleaned_list = remove_unwanted(f)
        list_elements = extract_list(cleaned_list)
        wrapped_list = wrap_details(cleaned_list)
    
    with open("clean.html", 'w') as fp:
        fp.write(wrapped_list)
        
    with open('clean.html', 'r') as fin:
        data = fin.read().splitlines(True)
    with open('clean.html', 'w') as fout:
        fout.writelines(data[1:])
        
def remove_unwanted(data):
    soup = BeautifulSoup(data, features="html.parser")
    soup = remove_class(soup)
    span_tag = soup.span.extract()
    nav_tag = soup.nav.extract()
    head_tag = soup.head.extract()
    return soup.prettify()

def extract_list(data):
    soup = BeautifulSoup(data, features="html.parser")
    list_tag = soup.ul.extract()
    return list_tag.prettify()

def wrap_details(data):
    soup = BeautifulSoup(data)

    ul_tag = soup.find("ul")
    div_tag = soup.new_tag("div")
    div_tag['style'] = "width: 800px; height: 400px; overflow-y: auto; background-color: #F1F3F4"
    ul_tag.wrap(div_tag)
    new_tag = soup.new_tag("details")
    div_tag.wrap(new_tag)
    
    tag = soup.new_tag("summary")
    tag.string = "Results"
    soup.div.insert_after(tag)

    return soup.prettify()

def remove_class(f):
    with open("output.html") as f:
        soup = BeautifulSoup(f, features="html.parser")

        for tag in soup():
            for attribute in ["class"]: # You can also add id,style,etc in the list
                del tag[attribute]

    return soup

## Endpoint Documentation
Endpoint documentation can be found by appending either `/docs` or `/redoc` on the end a base URL.

For example:
<ul>
  <li><a href="https://prov-api.testing.rrap-is.com//redoc" target="_blank">Redoc FastAPI</a></li>
  <li><a href="https://prov-api.testing.rrap-is.com//docs" target="_blank">Docs FastAPI</a> </li>
</ul>

Then select from the menu an endpoint function call e.g. `/explore/lineage`

Then append the function call onto the base url e.g. `https://prov-api.testing.rrap-is.com/explore/lineage`

[Return to Top](#toc)

## Demonstration

### Register a model run
see <a href="https://prov-api.testing.rrap-is.com/redoc#tag/Model-Runs/operation/register_model_run_complete_model_run_register_complete_post" target="_blank">Endpoint documentation</a>

In [17]:
auth = token_manager.get_auth
postfix = "/model_run/register_complete"
payload = {
  "inputs": {
    "datasets": [
      {
        "id": "10378.1/1687286"
      }
    ],
    "config_files": [
      {
        "id": "10378.1/1687612"
      }
    ],
    "parameter_files": [
      {
        "id": "10378.1/1687695"
      }
    ]
  },
  "outputs": {
    "datasets": [
      {
        "id": "10378.1/1687280"
      }
    ]
  },
  "associations": {
    "model": {
      "id": "10378.1/1687693"
    },
    "modeller": {
      "id": "10378.1/1687696"
    },
    "requesting_organisation": {
      "id": "10378.1/1687697"
    }
  }
}
endpoint = prov_api + postfix 

# When making the request, use auth=auth() - this will ensure tokens are valid
# right at the point of using them, including potentially auto refreshing!
response = requests.post(endpoint, json=payload, auth=auth())

convert_json_to_html(response.json())
HTML('clean.html')

### Explore lineage

In [20]:

auth = token_manager.get_auth
postfix = "/explore/lineage"
params = {
    "starting_id": "10378.1/1687588",
    "depth": 10
}
endpoint = prov_api + postfix

response = requests.get(endpoint, params=params, auth=auth())

convert_json_to_html(response.json())
HTML('clean.html')

result_graph = response.json()["graph"]

G = json_graph.node_link_graph(result_graph)

# Compute positions for viz.
pos = nx.spring_layout(G)

# Draw the graph using Altair
viz = nxa.draw_networkx(G, pos=pos)

# Show it as an interactive plot!
viz.interactive()

Token validation failed due to error: Signature has expired.
Refreshing using refresh token



alt.LayerChart(...)

## Explore Children

In [21]:
auth = token_manager.get_auth
postfix = "/explore/children"
params = {
    "starting_id": "10378.1/1687588"
}
endpoint = prov_api + postfix 

response = requests.get(endpoint, params=params, auth=auth())

convert_json_to_html(response.json())
HTML('clean.html')

result_graph = response.json()["graph"]

G = json_graph.node_link_graph(result_graph)

# Compute positions for viz.
pos = nx.spring_layout(G)

# Draw the graph using Altair
viz = nxa.draw_networkx(G, pos=pos,  arrow_width=5,  arrow_length=0.1, arrow_color='green')

# Show it as an interactive plot!
viz.interactive()

C:\office12\rrap-demo-blog\.mvp_demo_venv\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedWriter name=4>
  return process_handler(cmd, _system_body)
C:\office12\rrap-demo-blog\.mvp_demo_venv\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  return process_handler(cmd, _system_body)
C:\office12\rrap-demo-blog\.mvp_demo_venv\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=6>
  return process_handler(cmd, _system_body)


alt.LayerChart(...)

[Return to Top](#toc)